# Shor's factoring algorithm
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Shor's general factoring algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), modified as in [[E24]](https://doi.org/10.1145/3655026), and with the classical post-processing in [[E21b]](https://doi.org/10.1007/s11128-021-03069-1) and [[E24]](https://doi.org/10.1145/3655026).

To start off, let us pick $n$ prime factors $p_i$ uniformly at random from the set of all $l$-bit prime factors, and $n$ exponents $e_i$ uniformly at random from $[1, e_{\max}] \cap \mathbb Z$, for $i \in [1, n] \cap \mathbb Z$, and take the product to form

$$N = \prod_{i=1}^n p_i^{e_i}.$$

To this end, we use the [<code>sample_l_bit_prime(l)</code>](../docs/math/primes/sample_l_bit_prime.md) and [<code>sample_integer(B)</code>](../docs/math/random/sample_integer.md) convenience functions provided by [Quaspy](https://github.com/ekera/quaspy).

In [ ]:
!pip3 install -q quaspy # Make sure that quaspy is installed.

In [1]:
from quaspy.math.primes import sample_l_bit_prime;
from quaspy.math.random import sample_integer;

from math import prod;

n = 10;
l = 1024;
e_max = 1;

factors = [];
primes = set();

while len(factors) < n:
  pi = sample_l_bit_prime(l);
  ei = sample_integer(e_max) + 1;

  if pi not in primes:
    primes.add(pi);
    factors.append([pi, ei]);

    print("Sampled p" + str(len(factors)) + " =", pi);

N = prod([pi ** ei for [pi, ei] in factors]);

print("\nComputed N =", N);

Sampled p1 = 143653370906962213832221125972548186993793147225256479101915841314636455570619793778626523047081772987291883621498124341980839900153849834901562866869712349175174132611303508678354099155334357961279919040223656876640707689000311229807373332776462112895002715119420109503534890125841668435156145503897816255463
Sampled p2 = 106153134784350358413063271386813501001164923544918597765901791886784397928018190335997299057888395430750831604751690469446090993345500125851704250449550116482655100367052555882649562794487519355617787511150151335198376269039380945828047567351164548239630158998133279153268206457289372717862632786392510114069
Sampled p3 = 148857569733497845863220175361223191316606910471581532450259055474768711371113107629205426044478994257558574595136991165136600511913029464646782231738570373037496688123539999718304459199896700908516381223386700474890146402393425991759002285418115786781334562664999448804559301844510515759781443207221888220733
Sampled p4 = 127464129907058416

## 1. Simulating order finding in $\mathbb Z_N^*$ exactly to sample $r$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) for exactly sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and returning its order without explicitly computing and returning $g$. This function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$.

Below, we use this function to simulate the initial classical pre-processing step of Shor's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$.

In [2]:
from quaspy.factoring.sampling import sample_r_given_N;

r = sample_r_given_N(N, factors);

print("Sampled r =", r);

Sampled r = 3633280426436058885391151710041964433952788896326905599968249447775459922845743039961387576215900863146361413705274818709140969701732001336274347716927178544032368261906091703918040936144115129573324493757393431178257238980142216839035250204319443199187413427751112328122579144070925142013014638867192789706604952496790803621416987738844018177034501100195328191036251379430229311575213026404365536231816764501438605615978554296324654708004137242406594916406698690504343446538833885134229146986552973121640571736503288827129834452799658860132338878379296455619646019391845482013819230253619011643754005599121708756817456101674800390594257956440248128388619883064434531787013258427947373485790115495416382428250280813609174191741429874025128413464607182949923606145794552267170137820344019924321643524983015284813663943471825532232174552331841654568437899315722146724820598047596301868541887508074414981567399196375293261640104579362267021398782431368148705638410760698122502522659573727665

### 1.1. Solving $r$ for the complete factorization of $N$
We may then use the [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $r$ for the factors of $N$ by using the post-processing from [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [3]:
from quaspy.factoring.general.postprocessing.ekera import solve_r_for_factors;

result = solve_r_for_factors(r, N);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("\n[ OK ] All prime factors of N were successfully recovered.");
else:
  print("\n[FAIL] The integer N was not completely factored.");

Solving for the prime factors yielded: {148857569733497845863220175361223191316606910471581532450259055474768711371113107629205426044478994257558574595136991165136600511913029464646782231738570373037496688123539999718304459199896700908516381223386700474890146402393425991759002285418115786781334562664999448804559301844510515759781443207221888220733, 127464129907058416852419862544206483858533127614682008274342461070998109271419538757945769429936138086254619957140745465270203917586825925647569295659605419357676733795147574470348046548264642162544402366259752825380320428341980294083966523305795128852682450340727440893984049238933040572610068173238150780573, 119195687971922021372621191918058656854608926398271465007706405405764738065574471569348699030790937620461898813512366013331472104865637642884426948120456066512227210065116929693142650859173542010450981698211698894742228497478450587004379236830271423868055749443652299272623654486186450881748620911153631093189, 143653370906962213832221125

## 2. Simulating the quantum part of Shor's order-finding algorithm to sample $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm exactly (up to arbitrary precision) for a given order $r$. Below, we use the this function to simulate running the algorithm for $r$ with a control register of $m + \ell$ bits.

This yields a frequency $j$ sampled from the probability distribution induced by the algorithm:

In [4]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

m = N.bit_length() - 1;
l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l);

print("Sampled j =", j);

Sampled j = 3160961445531119556362965420807833999374700957411031200654687182663060613987651314002548228811683536204147728357433187924295281432472716493529402843679653944565220812485775295582002246658927980399253682030559880936352054705396152287121557783407831473607066207309361269342837503696532313813567368585544848559799782320876707169188235802012880524125291020684633958487706675423410577525010193321335248901923680443408569560799181428950356287236228100503890224348902188372469588796219591759727423169531719057054488434577846835534589724771404308631841363932050310009334388818715265425680635884397804639910183364673395056533208249520337629366630810025452712521657407719536331091392074550134630036934315964909197074055189716487369659992949580968776794281638832520378407419702241017071365035433641184599352858191450725206313268840460483164441343965191634324392890813851191134857396882245284056572408750987118629143356360968421508005723512832199885804157707585876079878643391798311547478508913719995

### 2.1. Solving $j$ for the complete factorization of $N$
We now proceed to solve $j$ for the factors of $N$.

To this end, we first use the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy) to define a group element $g$ of order $r$:

In [5]:
from quaspy.math.groups import SimulatedCyclicGroupElement;

g = SimulatedCyclicGroupElement(r);

We may then e.g. use the [<code>solve_j_for_factors(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_factors.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026), and to then solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [6]:
from quaspy.factoring.general.postprocessing.ekera import solve_j_for_factors;

result = solve_j_for_factors(j, m, l, g, N);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("\n[ OK ] All prime factors of N were successfully recovered.");
else:
  print("\n[FAIL] Failed to completely factor the integer N.");

Solving for the prime factors yielded: {148857569733497845863220175361223191316606910471581532450259055474768711371113107629205426044478994257558574595136991165136600511913029464646782231738570373037496688123539999718304459199896700908516381223386700474890146402393425991759002285418115786781334562664999448804559301844510515759781443207221888220733, 127464129907058416852419862544206483858533127614682008274342461070998109271419538757945769429936138086254619957140745465270203917586825925647569295659605419357676733795147574470348046548264642162544402366259752825380320428341980294083966523305795128852682450340727440893984049238933040572610068173238150780573, 119195687971922021372621191918058656854608926398271465007706405405764738065574471569348699030790937620461898813512366013331472104865637642884426948120456066512227210065116929693142650859173542010450981698211698894742228497478450587004379236830271423868055749443652299272623654486186450881748620911153631093189, 143653370906962213832221125

We may also solve in two steps, by first calling [<code>solve_j_for_r(j, m, l, g, ..)</code>](../docs/orderfinding/general/postprocessing/ekera/solve_j_for_r.md) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026), and by then calling [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) to solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [7]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r;
from quaspy.factoring.general.postprocessing.ekera import solve_r_for_factors;

rp = solve_j_for_r(j, m, l, g, accept_multiple=True);

print("Solving j for a multiple r' of the order r yielded:", rp);

if (rp != 1) and (r % rp == 0):
  print("\n[ OK ] A multiple r' of the order r was successfully recovered.");

  result = solve_r_for_factors(r, N);

  print("\nSolving r for the complete factorization of N yielded:", result);

  if result == primes:
    print("\n[ OK ] All prime factors of N were successfully recovered.");
  else:
    print("\n[FAIL] Failed to completely factor N.");

else:
  print("\n[FAIL] Failed to recover a multiple r' of the order r.");

Solving j for a multiple r' of the order r yielded: 363328042643605888539115171004196443395278889632690559996824944777545992284574303996138757621590086314636141370527481870914096970173200133627434771692717854403236826190609170391804093614411512957332449375739343117825723898014221683903525020431944319918741342775111232812257914407092514201301463886719278970660495249679080362141698773884401817703450110019532819103625137943022931157521302640436553623181676450143860561597855429632465470800413724240659491640669869050434344653883388513422914698655297312164057173650328882712983445279965886013233887837929645561964601939184548201381923025361901164375400559912170875681745610167480039059425795644024812838861988306443453178701325842794737348579011549541638242825028081360917419174142987402512841346460718294992360614579455226717013782034401992432164352498301528481366394347182553223217455233184165456843789931572214672482059804759630186854188750807441498156739919637529326164010457936226702139878243136

## 3. Simulating order finding in $\mathbb Z_N^*$ heuristically to sample $g$ and $r$

In addition to the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function used above, [Quaspy](https://github.com/ekera/quaspy) also provides a function [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) for sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and heuristically computing and returning its order $r$ alongside $g$.

This function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$. (If the function is also fed the factorization of $p_i - 1$ for $i \in [1, n] \cap \mathbb Z$ then the function can be made exact, as explained in the [documentation](../docs/factoring/sampling/sample_r_given_N.md).)

Below, we use this function to simulate the initial classical pre-processing step of Shor's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$. This yields $g$ and $r$:

In [8]:
from quaspy.factoring.sampling import sample_g_r_given_N;

[g, r] = sample_g_r_given_N(N, factors);

print("Sampled g =", g, "\n");
print("Sampled r =", r, "\n");

Sampled g = 1956784174782571763124417858413159841782231448975404652285046468985053650796816292135414151692805034547734824940562834816531148729196930660385167999687024324301549538986131664613213854542579058624320099944365518681984629502886306874482335568228244448630221539575092628804213620639613669890242727813260696401780665078822742813021702391277848571410419083108573083612603410198625739336663546021566954367969640842723478223372009299835000131914347702141219709874473929007439757755789480832333869367117215279128072620195108232568550692654989249278817841648832878267624909023272361966257253579600879787170115789271176894645267050603216415744432799946537814430439049522098743836927071510060712085577213863825611157685049691752507960238616463915038942491670085048968377388939588080944858125629915174778709666299650473167690003354094997673284663322043376393336333181543736022454637923030031469450760833278039054358483813710230123658643988350055323324552169123350923366880290829488256087880417362520

### 3.1. Simulating the quantum part of Shor's order-finding algorithm to sample $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm exactly (up to arbitrary precision) for a given order $r$. Below, we use the this function to simulate running the algorithm for $r$ with a control register of $m + \ell$ bits.

This yields a frequency $j$ sampled from the probability distribution induced by the algorithm:

In [9]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

m = N.bit_length() - 1;
l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l);

print("Sampled j =", j);

Sampled j = 7976221141965341281834581448067719801998867919607648599741778904333076082121969877838510524497795847673984887041839594918178543932502080087789869338719465746196824707316274026271164979495549669796557101037527643138027405852595700785719878818708940105392929368733835346076588233030363301242661480786665820625648913841860081048702680468609499921059386620454505759126751337730730850808097577002708488356191909934117070919764538997796627455033987300385545887297187038068737153261014733322898290493238858428754476515523589927143185328585241907323127303352118002338606924634169298735589382187579514472547975486296970109524904743785493144861501062236765702545977577115379183089537794922936859604233509446110685331429436801048957759158913256753634145663048276250258014420655992095775340593038487438709880963506589105722462905220551494880551603670120358982259020384145979661681276775572848535812492465435851109138034737796409801842777313632793871204736148981502123069630414723689450618623782832552

### 3.2. Solving $j$ for the complete factorization of $N$
We may then e.g. use the [<code>solve_j_for_factors_mod_N(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_factors_mod_N.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026), and to then solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [10]:
from quaspy.factoring.general.postprocessing.ekera import solve_j_for_factors_mod_N;

result = solve_j_for_factors_mod_N(j, m, l, g, N);

print("Solving for all prime factors of N via [E21b] yielded:", result);

if result == primes:
  print("\n[ OK ] All prime factors of N were successfully recovered.");
else:
  print("\n[FAIL] Failed to completely factor the integer N.");

Solving for all prime factors of N via [E21b] yielded: {148857569733497845863220175361223191316606910471581532450259055474768711371113107629205426044478994257558574595136991165136600511913029464646782231738570373037496688123539999718304459199896700908516381223386700474890146402393425991759002285418115786781334562664999448804559301844510515759781443207221888220733, 127464129907058416852419862544206483858533127614682008274342461070998109271419538757945769429936138086254619957140745465270203917586825925647569295659605419357676733795147574470348046548264642162544402366259752825380320428341980294083966523305795128852682450340727440893984049238933040572610068173238150780573, 119195687971922021372621191918058656854608926398271465007706405405764738065574471569348699030790937620461898813512366013331472104865637642884426948120456066512227210065116929693142650859173542010450981698211698894742228497478450587004379236830271423868055749443652299272623654486186450881748620911153631093189, 14365337090

### 3.3. Solving $j$ for $r$
We may also use the use the [<code>solve_j_for_r_mod_N(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_r_mod_N.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $j$ for $r$ using the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026).

In [11]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r_mod_N;

result = solve_j_for_r_mod_N(j, m, l, g, N);

print("Solving j for r via [E22b] yielded:", result);

if result == r:
  print("\n[ OK ] The order r was successfully recovered.");
else:
  print("\n[FAIL] Failed to recover the order r.");

Solving j for r via [E22b] yielded: 9991521172699161934825667202615402193370169464898990399912685981382514787825793359893815834593727373652493887689505751450137666679763003674754456221549740996089012720241752185774612574396316606326642357832831935740207407195391096307346938061878468797765386926315558902337092646195044140535790256884780171693163619366174709958896716281821049986844878025537152525349691293433130606831835822612005224637496102378956165443941024314892800447011377416618136020118421398886944477981793184119130154213020676084511572275384044274607044745199061865363931915543065252954026553327575075538002883197452282020323515397584699081248004279605701074134209380210682353068704678427194962414286460676855277085922817612395051677688272237425229027288932153569103137027669753112289916900935018734717879005946054791884519693703292033237575844547520213638480018912564550063204223118235903493256644630889830138490190647204641199310347790032056469510287593246234308846651686262408940505629591

### 3.4. Splitting $N$ given $g$ and $r$ using Shor's original algorithm

We may then split $N$ given $g$ and $r$ using Shor's original algorithm in [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700).

To this end, we use the [<code>split_N_given_g_r(g, r, N)</code>](../docs/factoring/general/postprocessing/shor/split_N_given_g_r.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [12]:
from quaspy.factoring.general.postprocessing.shor import split_N_given_g_r;

result = split_N_given_g_r(g, r, N);

print("Attempting to split N using Shor's algorithm yielded:", result);

Attempting to split N using Shor's algorithm yielded: {405845698954978023777115984221813761162044717496005602587484779494352691667208409981644347156357879204481021403785722167755824439066543563730004401804321452482491057891434042265849201807220984512740697460958343906226678877050096229841237060823065089982984664416521183060578816679980361022719149781011438264877772883426890140216209203340307119638998735395540102300992625943957586766903505133196137115832554397394578042335749897990509720172969404695216837322065697239688363221258760638829146151171959045102772670708656794712183300747222186568888336249675280209277392753044278873070417004002145328617251796642212436150928206231125342519187115373601178721699556678802059381900526182727413667319212192688313433973442436442966391256196730127488588848434433604239025145732861886538642988533805195718352057403333604982269732137266029157159468923677794925648262774238414702933072382953445407526314663705543042894302913203753959557369649765112428853984990